In [1]:
from __future__ import division, print_function

In [2]:
# from seqdataloader.batchproducers import coordbased
from seqdataloader.batchproducers import coordbased
import gzip
import numpy as np

class ColsInBedFile(
    coordbased.coordstovals.core.AbstractSingleNdarrayCoordsToVals):
    def __init__(self, gzipped_bed_file, **kwargs):
        super(ColsInBedFile, self).__init__(**kwargs)
        self.gzipped_bed_file = gzipped_bed_file
        coords_to_vals = {}
        for row in gzip.open(gzipped_bed_file, 'rb'):
            row = row.decode("utf-8").rstrip()
            split_row = row.split("\t")
            chrom_start_end = split_row[0]+":"+split_row[1]+"-"+split_row[2]
            vals = np.array([float(x) for x in split_row[4:]])
            coords_to_vals[chrom_start_end] = vals
        self.coords_to_vals = coords_to_vals
        
    def _get_ndarray(self, coors):
        to_return = []
        for coor in coors:
            chrom_start_end = (coor.chrom+":"
                               +str(coor.start)+"-"+str(coor.end))
            to_return.append(self.coords_to_vals[chrom_start_end])
        return np.array(to_return)
    
    
inputs_coordstovals = coordbased.coordstovals.fasta.PyfaidxCoordsToVals(
  genome_fasta_path='/mnt/data/annotations/by_release/hg38/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta',
  center_size_to_use=1000)

targets_coordstovals = ColsInBedFile(
       gzipped_bed_file="summits_with_signal.bed.gz")
            
keras_train_batch_generator = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer=coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
      bed_file="train_summits_with_signal.bed.gz",
      coord_batch_transformer=coordbased.coordbatchtransformers.ReverseComplementAugmenter(),
      batch_size=128,
      shuffle_before_epoch=True,
      seed=1234
    ),
    inputs_coordstovals=inputs_coordstovals,
    targets_coordstovals=targets_coordstovals
)


keras_valid_batch_generator = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer = coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
        bed_file="valid_summits_with_signal.bed.gz",
        coord_batch_transformer=coordbased.coordbatchtransformers.ReverseComplementAugmenter(),
        batch_size= 64, 
        shuffle_before_epoch=True, 
        seed=1234
    ),
    inputs_coordstovals=inputs_coordstovals, 
    targets_coordstovals=targets_coordstovals
)

keras_test_batch_generator = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer = coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
        bed_file="test_summits_with_signal.bed.gz",
        coord_batch_transformer=coordbased.coordbatchtransformers.ReverseComplementAugmenter(),
        batch_size = 64, 
        shuffle_before_epoch = True, 
        seed = 1234
    ), 
    inputs_coordstovals = inputs_coordstovals, 
    targets_coordstovals = targets_coordstovals
)

Using TensorFlow backend.


In [3]:
y_test = np.array([val for batch in keras_test_batch_generator for val in batch[1]], dtype = 'float32') 

In [4]:
import keras 
import keras_genomics
import numpy as np
import keras.layers as k1

from keras import backend as K 
from keras.layers.core import Dropout 
from keras.layers.core import Flatten
from keras.layers import Input
from keras.engine import Layer
from keras.models import Sequential 
from keras.engine.base_layer import InputSpec
from keras.models import Model
from keras.models import load_model

In [5]:
class RevCompSumPool(Layer): 
    def __init__(self, **kwargs): 
        super(RevCompSumPool, self).__init__(**kwargs)

    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(RevCompSumPool, self).build(input_shape)

    def call(self, inputs): 
        #divide by sqrt 2 for variance preservation
        inputs = (inputs[:,:,:int(self.num_input_chan/2)] + inputs[:,:,int(self.num_input_chan/2):][:,::-1,::-1])/(1.41421356237)
        return inputs
      
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], int(input_shape[2]/2))


In [6]:
kernel_size = 15
filters= 15
input_length = 1000

from numpy.random import seed
from tensorflow import set_random_seed
from keras.callbacks import EarlyStopping, History, ModelCheckpoint

seed_num = 1000
seed(seed_num)
set_random_seed(seed_num)

In [7]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [8]:
scale = 1.0
rc_model_standard = keras.models.Sequential()
rc_model_standard.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, 
            input_shape=keras_train_batch_generator[0][0].shape[1:], padding="same"))
# rc_model_standard.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard.add(k1.core.Activation("relu"))
# rc_model_standard.add(keras_genomics.layers.RevCompConv1D(
#             filters=filters, kernel_size=kernel_size, padding="same"))
# rc_model_standard.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
# rc_model_standard.add(k1.core.Activation("relu"))
# rc_model_standard.add(keras_genomics.layers.RevCompConv1D(
#             filters=filters, kernel_size=kernel_size,padding="same"))
# rc_model_standard.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
# rc_model_standard.add(k1.core.Activation("relu"))
rc_model_standard.add(RevCompSumPool())
rc_model_standard.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same", strides=40))
rc_model_standard.add(Flatten())
# rc_model_standard.add(keras_genomics.layers.core.Dense(units = 100, activation = "relu"))
rc_model_standard.add(keras_genomics.layers.core.Dense(units = 1))

rc_model_standard.compile(optimizer="adam", loss='mean_squared_error')
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 60,
                              restore_best_weights=True)

history_rc_standard = rc_model_standard.fit_generator(generator=keras_train_batch_generator, 
                                                      epochs=300, callbacks = [early_stopping_callback],
                                                     validation_data=keras_valid_batch_generator)
    
rc_model_standard.set_weights(early_stopping_callback.best_weights)

W0716 09:00:15.683277 140481344096000 deprecation_wrapper.py:119] From /users/hannahgz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0716 09:00:15.934597 140481344096000 deprecation_wrapper.py:119] From /users/hannahgz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0716 09:00:15.938580 140481344096000 deprecation_wrapper.py:119] From /users/hannahgz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0716 09:00:16.002102 140481344096000 deprecation_wrapper.py:119] From /users/hannahgz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d inste

Epoch 1/300
285/285 [==============================] - 81s 284ms/step - loss: 11131.3158 - val_loss: 10170.2470
Epoch 2/300
285/285 [==============================] - 84s 293ms/step - loss: 8567.7840 - val_loss: 10113.7754
Epoch 3/300
285/285 [==============================] - 74s 261ms/step - loss: 8562.2848 - val_loss: 10078.1565
Epoch 4/300
285/285 [==============================] - 89s 312ms/step - loss: 8545.8021 - val_loss: 10125.2671
Epoch 5/300
285/285 [==============================] - 81s 283ms/step - loss: 8544.5493 - val_loss: 10101.7727
Epoch 6/300
285/285 [==============================] - 69s 242ms/step - loss: 8525.6470 - val_loss: 10094.4560
Epoch 7/300
285/285 [==============================] - 80s 282ms/step - loss: 8505.3470 - val_loss: 10172.0569
Epoch 8/300
285/285 [==============================] - 92s 324ms/step - loss: 8471.6838 - val_loss: 10064.0465
Epoch 9/300
285/285 [==============================] - 88s 309ms/step - loss: 8428.9663 - val_loss: 10057.1579


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



285/285 [==============================] - 61s 215ms/step - loss: 5297.1101 - val_loss: 6459.7513
Epoch 205/300
285/285 [==============================] - 53s 186ms/step - loss: 5298.1708 - val_loss: 6501.8065
Epoch 206/300
285/285 [==============================] - 55s 192ms/step - loss: 5296.5015 - val_loss: 6488.1603
Epoch 207/300
285/285 [==============================] - 56s 195ms/step - loss: 5296.3719 - val_loss: 6440.4180
Epoch 208/300
285/285 [==============================] - 55s 193ms/step - loss: 5296.8318 - val_loss: 6465.3300
Epoch 209/300
285/285 [==============================] - 54s 190ms/step - loss: 5301.2492 - val_loss: 6452.2312
Epoch 210/300
285/285 [==============================] - 58s 205ms/step - loss: 5301.8036 - val_loss: 6453.5208
Epoch 211/300
285/285 [==============================] - 58s 202ms/step - loss: 5291.4373 - val_loss: 6448.0979
Epoch 212/300
285/285 [==============================] - 58s 202ms/step - loss: 5297.4432 - val_loss: 6478.5101
Epoch 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
rc_standard_filename = ('rc_standard_%s.h5' % seed_num, str(seed_num))[0]
rc_model_standard.save(rc_standard_filename)
custom_objects = {'RevCompConv1D':keras_genomics.layers.RevCompConv1D, 
                  'RevCompSumPool':RevCompSumPool}
rc_standard_model_final = load_model(rc_standard_filename, custom_objects)

In [ ]:
from keras.initializers import Initializer
def _compute_fans(shape, data_format='channels_last'):
    """Computes the number of input and output units for a weight shape.
    # Arguments
        shape: Integer shape tuple.
        data_format: Image data format to use for convolution kernels.
            Note that all kernels in Keras are standardized on the
            `channels_last` ordering (even when inputs are set
            to `channels_first`).
    # Returns
        A tuple of scalars, `(fan_in, fan_out)`.
    # Raises
        ValueError: in case of invalid `data_format` argument.
    """
    if len(shape) == 2:
        fan_in = shape[0]
        fan_out = shape[1]
    elif len(shape) in {3, 4, 5}:
        # Assuming convolution kernels (1D, 2D or 3D).
        # TH kernel shape: (depth, input_depth, ...)
        # TF kernel shape: (..., input_depth, depth)
        if data_format == 'channels_first':
            receptive_field_size = np.prod(shape[2:])
            fan_in = shape[1] * receptive_field_size
            fan_out = shape[0] * receptive_field_size
        elif data_format == 'channels_last':
            receptive_field_size = np.prod(shape[:-2])
            fan_in = shape[-2] * receptive_field_size
            fan_out = shape[-1] * receptive_field_size
        else:
            raise ValueError('Invalid data_format: ' + data_format)
    else:
        # No specific assumptions.
        fan_in = np.sqrt(np.prod(shape))
        fan_out = np.sqrt(np.prod(shape))
    return fan_in, fan_out

class RevcompVarianceScaling(Initializer):
    def __init__(self, scale=1.0,
                 mode='fan_in',
                 distribution='normal',
                 seed=None):
        if scale <= 0.:
            raise ValueError('`scale` must be a positive float. Got:', scale)
        mode = mode.lower()
        if mode not in {'fan_in', 'fan_out', 'fan_avg'}:
            raise ValueError('Invalid `mode` argument: '
                             'expected on of {"fan_in", "fan_out", "fan_avg"} '
                             'but got', mode)
        distribution = distribution.lower()
        if distribution not in {'normal', 'uniform'}:
            raise ValueError('Invalid `distribution` argument: '
                             'expected one of {"normal", "uniform"} '
                             'but got', distribution)
        self.scale = scale
        self.mode = mode
        self.distribution = distribution
        self.seed = seed

    def __call__(self, shape, dtype=None):
        fan_in, fan_out = _compute_fans(shape)
        fan_out = fan_out*2 #revcomp kernel underestimates fan_out
        print("fanin:",fan_in, "fanout:",fan_out, self.scale, self.mode)
        scale = self.scale
        if self.mode == 'fan_in':
            scale /= max(1., fan_in)
        elif self.mode == 'fan_out':
            scale /= max(1., fan_out)
        else:
            scale /= max(1., float(fan_in + fan_out) / 2)
        if self.distribution == 'normal':
            # 0.879... = scipy.stats.truncnorm.std(a=-2, b=2, loc=0., scale=1.)
            stddev = np.sqrt(scale) / .87962566103423978
            return K.truncated_normal(shape, 0., stddev,
                                      dtype=dtype, seed=self.seed)
        else:
            limit = np.sqrt(3. * scale)
            return K.random_uniform(shape, -limit, limit,
                                    dtype=dtype, seed=self.seed)

    def get_config(self):
        return {
            'scale': self.scale,
            'mode': self.mode,
            'distribution': self.distribution,
            'seed': self.seed
        }

In [ ]:
rc_model_var = keras.models.Sequential()
rc_model_var.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, 
            input_shape=keras_train_batch_generator[0][0].shape[1:], padding="same", 
            kernel_initializer = RevcompVarianceScaling(
                                 scale= scale,
                                 mode='fan_avg',
                                 distribution='uniform',
                                 seed=None)))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_var.add(k1.core.Activation("relu"))
# rc_model_var.add(keras_genomics.layers.RevCompConv1D(
#             filters=filters, kernel_size=kernel_size, padding="same", 
#             kernel_initializer = RevcompVarianceScaling(
#                                  scale= scale,
#                                  mode='fan_avg',
#                                  distribution='uniform',
#                                  seed=None)))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
# rc_model_var.add(k1.core.Activation("relu"))
# rc_model_var.add(keras_genomics.layers.RevCompConv1D(
#             filters=filters, kernel_size=kernel_size,padding="same", 
#             kernel_initializer = RevcompVarianceScaling(
#                                  scale= scale,
#                                  mode='fan_avg',
#                                  distribution='uniform',
#                                  seed=None)))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
# rc_model_var.add(k1.core.Activation("relu"))
rc_model_var.add(RevCompSumPool())
rc_model_var.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same", strides=40))
rc_model_var.add(Flatten())
# rc_model_var.add(keras_genomics.layers.core.Dense(units = 100, activation = "relu", 
#                                              kernel_initializer = RevcompVarianceScaling(
#                                                  scale= scale,
#                                                  mode='fan_avg',
#                                                  distribution='uniform',
#                                                  seed=None)))
rc_model_var.add(keras_genomics.layers.core.Dense(units = 1))


rc_model_var.compile(optimizer="adam", loss='mean_squared_error')
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 60,
                              restore_best_weights=True)

history_rc_var = rc_model_var.fit_generator(generator=keras_train_batch_generator, 
                                                      epochs=200, callbacks = [early_stopping_callback],
                                                     validation_data=keras_valid_batch_generator)
    
rc_model_var.set_weights(early_stopping_callback.best_weights)

In [ ]:
rc_var_filename = ('rc_var_%s.h5' % seed_num, str(seed_num))[0]
rc_model_var.save(rc_var_filename)
custom_objects = {'RevCompConv1D':keras_genomics.layers.RevCompConv1D,
                  'RevCompSumPool':RevCompSumPool, 
                 'RevcompVarianceScaling':RevcompVarianceScaling}
rc_var_model_final = load_model(rc_var_filename, custom_objects)

In [ ]:
y_pred_standard = rc_model_standard.predict_generator(keras_test_batch_generator)
y_pred_var = rc_model_var.predict_generator(keras_test_batch_generator)

In [ ]:
from matplotlib import pyplot as plt
from scipy.stats import spearmanr

plt.scatter(y_test, y_pred_standard, alpha = 0.1)
plt.xlabel("True Labels")
plt.ylabel("Predicted Labels")
plt.show()
print(spearmanr(y_test, y_pred_standard))

plt.scatter(y_test, y_pred_var, alpha = 0.1)
plt.xlabel("True Labels")
plt.ylabel("Predicted Labels")
# plt.plot([np.min(y_test), np.max(y_test)],
#          [np.min(y_pred), np.max(y_test)],
#           color="black")
plt.show()
print(spearmanr(y_test, y_pred_var))